# Processamento de linguagem natural para modelo de busca por voz.

# 1.0 Instalação

## 1.1 Instalação da bibliotecas

In [ ]:
!pip install unidecode
!pip install spacy
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.8 MB/s eta 0:00:00


## 1.2 Importação de bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from unidecode import unidecode

import spacy

#nltk para pre proscesamento e tokenização
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re

from nltk.stem import WordNetLemmatizer

nltk.download('rslp')

stemmer = nltk.stem.RSLPStemmer()

# Pacote nlp para portugues
nlp = spacy.cli.download('pt_core_news_sm')
nlp = spacy.load('pt_core_news_sm')

# Pacote nlp para portugues
# nlp = spacy.cli.download('pt_core_news_lg')
# nlp = spacy.load('pt_core_news_lg')

# Pacote nlp para ingles
# nlp = spacy.cli.download('en_core_web_sm')
# nlp = spacy.load("en_core_web_sm")


import requests
#Biblioteca usada para consultar uma URL
import urllib.request
from urllib.request import urlopen

#Biblioteca usada analisar os dados retornados da url
from bs4 import BeautifulSoup

#importação de keras, para execução do modelo e criação do dicionario
from keras.preprocessing.text import Tokenizer

# Google Drive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Mounted at /content/drive


# 2.0 Importação e compreensão dos dados

In [ ]:
df_url = pd.read_csv('/content/drive/Shareddrives/CogniVoice/ibm_db.csv')

In [ ]:
df_url

,url_noticia
0,https://www.showmetech.com.br/ibm-mostra-avanc...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...
2,https://www.convergenciadigital.com.br/Negocio...
3,https://inforchannel.com.br/2023/08/19/intelig...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...
5,https://www.tecmundo.com.br/voxel/219536-overw...
6,https://epocanegocios.globo.com/futuro-do-dinh...
7,https://tecmasters.com.br/ceo-openai-diretora-...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...
9,https://olhardigital.com.br/2023/08/22/ciencia...


In [ ]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url_noticia  20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


# 3.0 Testes TDD

# 4.0 Tratamento dos dados

In [ ]:
dados_dataframe = [
    "Esta é a primeira frase.",
    "Aqui vem a segunda frase com palavras de parada.",
    "E a terceira frase.",
    "Quarta frase é necessário o uso de acentos e pontuações",
    "Teste para pontuação (virgulas, `""´ e dois pontos:)",
    "Exemplo de titulo 1  - Inforchannel",
    "Exemplo de titulo 2 | Voxel",
    "Exemplo de titulo 3 Por Investing.com",
    "  Isso  é um   exemplo   ",
    "  Espaços   múltiplos     ",
    "oi, tudo 3 bem 4 com voce?",
    "isso 76 apresenta 12 número isolados"
]

test_dataframe = pd.DataFrame(dados_dataframe)


maisculas_test = [
    ["OI", "TESTE", "maisculas", "Funcionando", "KKKKKKK", "Risadas"]
]

maisculas_dataframe = pd.DataFrame(maisculas_test)


test_dataframe

,0
0,Esta é a primeira frase.
1,Aqui vem a segunda frase com palavras de parada.
2,E a terceira frase.
3,Quarta frase é necessário o uso de acentos e p...
4,"Teste para pontuação (virgulas, `´ e dois pont..."
5,Exemplo de titulo 1 - Inforchannel
6,Exemplo de titulo 2 | Voxel
7,Exemplo de titulo 3 Por Investing.com
8,Isso é um exemplo
9,Espaços múltiplos


In [ ]:
maisculas_dataframe

,0,1,2,3,4,5
0,OI,TESTE,maisculas,Funcionando,KKKKKKK,Risadas


## 4.1 Remoção de acentos

### 4.1.1 Definição da função

In [ ]:
def remocaoAcentos(text):
    return unidecode(text)

### 4.1.2 Teste da função

In [ ]:
test_dataframe = test_dataframe.applymap(remocaoAcentos)

## 4.2 Tratamento de letras maiusculas

### 5.2.1 Definição da função

In [ ]:
def maiusculas(dados):
    if isinstance(dados, pd.DataFrame):
        return dados.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    elif isinstance(dados, str):
        return dados.lower()
    else:
        return dados

def lowercase_text(text):
  return text.lower()

### 5.2.2 Teste da função

In [ ]:
test_dataframe = test_dataframe.applymap(lowercase_text)
maisculas_dataframe = maisculas_dataframe.applymap(lowercase_text)

In [ ]:
test_dataframe

,0
0,esta e a primeira frase.
1,aqui vem a segunda frase com palavras de parada.
2,e a terceira frase.
3,quarta frase e necessario o uso de acentos e p...
4,"teste para pontuacao (virgulas, `' e dois pont..."
5,exemplo de titulo 1 - inforchannel
6,exemplo de titulo 2 | voxel
7,exemplo de titulo 3 por investing.com
8,isso e um exemplo
9,espacos multiplos


In [ ]:
maisculas_dataframe

,0,1,2,3,4,5
0,oi,teste,maisculas,funcionando,kkkkkkk,risadas


## 4.3 Tratamento do veiculo da noticia

### 4.3.1 Definição da função

In [ ]:
def clean_title(match_comp):
    pattern = re.compile(r'^(.*?)\s*(-|\||por)\s*.*$')
    match = pattern.match(match_comp)
    return match.group(1) if match else match_comp

### 4.3.2 Teste da função

In [ ]:
test_dataframe[0] = test_dataframe[0].apply(clean_title)

## 4.4 Limpeza pontuação

Parâmetro [^\w\s]|_ para localizar e substituir caracteres não alfanuméricos em uma string.

### 4.4.1 Definição da função

In [ ]:
def remove_marks(text):
    if isinstance(text, str): #verifica se o que está sendo processado é uma string
        return re.sub(r'[^\w\s]|_', ' ', text) #substituir todos os caracteres especiais por uma string vazia
    else:
        return text

### 4.4.2 Teste da função

In [ ]:
test_dataframe[0] = test_dataframe[0].apply(remove_marks)

## 4.5 Remoção de aspas

### 4.5.1 Definição da função

In [ ]:
def remove_aspas(column_or_text):
    if isinstance(column_or_text, pd.Series):  # Verifica se é uma coluna de DataFrame
        return column_or_text.apply(remove_aspas)
    elif isinstance(column_or_text, str):  # Verifica se é uma string
        return column_or_text.replace('"', '')
    else:
        return column_or_text  # Retorna outros tipos de dados sem modificação

### 4.6.2 Teste da função

In [ ]:
test_dataframe = test_dataframe.apply(remove_aspas)

## 4.6 Remoção de espaços duplicados

### 4.6.1 Definição da função

In [ ]:
def remove_duplicate_spaces(text):
    # Substitui sequências de espaços em branco por um único espaço
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

### 4.6.2 Teste da função

In [ ]:
test_dataframe[0] = test_dataframe[0].apply(remove_duplicate_spaces)

## 4.7 Remoção de números isolados

### 4.7.1 Definição da função

In [ ]:
def remove_isolated_numbers(text):
    # Substitui números isolados por um espaço em branco
    cleaned_text = re.sub(r'\b\d+\b', '', text)
    return cleaned_text.strip()

### 4.7.2 Definição da função

In [ ]:
test_dataframe[0] = test_dataframe[0].apply(remove_isolated_numbers)

## 4.8 Dataframe resultante da seção

In [ ]:
test_dataframe

,0
0,esta e a primeira frase
1,aqui vem a segunda frase com palavras de parada
2,e a terceira frase
3,quarta frase e necessario o uso de acentos e p...
4,teste para pontuacao virgulas e dois pontos
5,exemplo de titulo
6,exemplo de titulo
7,exemplo de titulo
8,isso e um exemplo
9,espacos multiplos


# 5.0 Web Scraping

## 5.1 Extração do titulo

### 5.1.1 Definição da função

In [ ]:
def extrair_titulo(url):
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
  }
  req = urllib.request.Request(url, headers=headers)
  page = urllib.request.urlopen(req)
  soup = BeautifulSoup(page, 'html.parser')
  title_new = soup.title.string
  title_new = clean_title(title_new)
  return title_new

### 5.1.2 Teste da função

In [ ]:
df_url['titulo'] = df_url['url_noticia'].apply(extrair_titulo)

## 5.2 Extração da noticia na integra e tratamento do dimensionamento

### 5.2.1 Definição de função

In [ ]:
def remove_tags(url):
  useless_tittle = extrair_titulo(url)

  useless_tittle = remocaoAcentos(useless_tittle)

  useless_tittle = maiusculas(useless_tittle)

  tittle = clean_title(useless_tittle)
  # parse html content
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
  }

  req = urllib.request.Request(url, headers=headers)
  page = urllib.request.urlopen(req)

  soup = BeautifulSoup(page, "html.parser")

  for data in soup(['style', 'script', 'title']):
    # Remove as tags
    data.decompose()

  # return data by retrieving the tag content
  all_text = ' '.join(soup.stripped_strings)
  # return all_text
  all_text = maiusculas(all_text)
  all_text = remocaoAcentos(all_text)


  quebra = all_text.find(tittle)

  if quebra != -1:  # verifica encontrou a quebra
    all_text = all_text[quebra:]
    all_text = remove_marks(all_text)
    all_text = remove_aspas(all_text)
    all_text = remove_duplicate_spaces(all_text)
    all_text = remove_isolated_numbers(all_text)
    return all_text
  else:
    all_text = remove_marks(all_text)
    all_text = remove_aspas(all_text)
    all_text = remove_duplicate_spaces(all_text)
    all_text = remove_isolated_numbers(all_text)
    return all_text

### 5.2.2 Teste da função

In [ ]:
df_url['tag_removed'] = df_url['url_noticia'].apply(remove_tags)

## 5.3 Dataframe resultante da seção

In [ ]:
df_url

,url_noticia,titulo,tag_removed
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...",ibm fala sobre avancos da watsonx plataforma d...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,ibm e microsoft expandem parceria para oferece...
2,https://www.convergenciadigital.com.br/Negocio...,IBM Brasil quer levar base instalada de Watson...,ibm brasil quer levar base instalada de watson...
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,inteligencia artificial da ibm comentara jogos...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,ibm quer ampliar base de clientes watson para ...
5,https://www.tecmundo.com.br/voxel/219536-overw...,Overwatch League traz Hero Rankings com a IA I...,overwatch league traz hero rankings com a ia i...
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,bradesco e itau se aliam a ibm para testar uso...
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,ceo da openai e diretora da ibm prestam depoim...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,ibm lancara llama da meta em plataforma de ia...
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,ibm lanca chip de ia analogico inspirado no ce...


# 6.0 Extração dos dados

## 6.1 Extração da data

### 6.1.1 Definição da função

In [ ]:
date_regex = [
  r'(\d{1,2} de [a-zA-Z]+ de \d{4})',
  r'(\d{1,2}/\d{1,2}/\d{2,4})',
  r'(\d{1,2} [a-zA-Z]+ \d{2,4})'
]

def extract_dates(row):
    quebra = row['tag_removed'].find(row['titulo'])
    for regex in date_regex:
        match = re.search(regex, row['tag_removed'][quebra:])
        if match:
            return match.group(1)
    return None

### 6.1.2 Teste da função

In [ ]:
df_url['data'] = df_url.apply(extract_dates, axis=1)

## 6.2 Extração da fonte

### 6.2.1 Definição da função

In [ ]:
def extrair_dominio(url):
    regex_url = r'https://(.*?)/'
    fonte = re.findall(regex_url, url)
    if fonte:
        return fonte[0]
    else:
        return None

### 6.2.2 Teste da função

In [ ]:
df_url['fonte'] = df_url['url_noticia'].apply(extrair_dominio)

## 6.3 Extração das entidades nomeadas

### 6.3.1 Definição da função

In [ ]:
def named_entities_filtred(coluna, filtro):
  results = []
  for linha in coluna:
    # print(linha)
    doc = nlp(linha)
    org_entities = set(ent.text for ent in doc.ents if ent.label_ == filtro)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    results.append(org_entities)
    # print("teste",results)
  return results

def named_entities(coluna):
  results = []
  for linha in coluna:
    # print(linha)
    doc = nlp(linha)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    results.append(entities)
    # print("teste",results)
  return results

### 6.3.2 Teste da função

In [ ]:
teste_entities_df = pd.DataFrame()

df_url["empresas"] = named_entities_filtred(df_url['tag_removed'], "ORG")
df_url["pessoas"] = named_entities_filtred(df_url['tag_removed'], "PER")
teste_entities_df["Entidades"] = named_entities(df_url['tag_removed'])

## 6.4 Dataframe resultante da seção

In [ ]:
df_url

,url_noticia,titulo,tag_removed,data,fonte,empresas,pessoas
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...",ibm fala sobre avancos da watsonx plataforma d...,None,www.showmetech.com.br,"{ibm, ibm brasil, halloween, bradesco, ibm ins...","{glauco vital, watsonx governance gestores, ma..."
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,ibm e microsoft expandem parceria para oferece...,None,inforchannel.com.br,"{azure, ibm, ibm consulting azure openai servi...","{golpes infoblox, julius baer group, francesco..."
2,https://www.convergenciadigital.com.br/Negocio...,IBM Brasil quer levar base instalada de Watson...,ibm brasil quer levar base instalada de watson...,None,www.convergenciadigital.com.br,"{ibm, ibm brasil, bradesco, watsonx, banco do ...","{foton, plataforma watson, watson assistant, m..."
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,inteligencia artificial da ibm comentara jogos...,None,inforchannel.com.br,"{ibm, ibm ix, ibm watsonx, infor channel, ibm ...","{red hat openshift, ibm watsonx powered, jonat..."
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,ibm quer ampliar base de clientes watson para ...,None,itforum.com.br,"{ibm, ibm brasil, ibm watsonx, ibm institute f...","{plataforma watson, computerworld cio, marcelo..."
5,https://www.tecmundo.com.br/voxel/219536-overw...,Overwatch League traz Hero Rankings com a IA I...,overwatch league traz hero rankings com a ia i...,None,www.tecmundo.com.br,"{blizzard, ibm, ibm voce, cupom nike, epic gam...","{tiktok twitter, thomas schulze costa, contato..."
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,bradesco e itau se aliam a ibm para testar uso...,None,epocanegocios.globo.com,"{ibm, editora globo, ibm brasil, bradesco, onu...","{wagner, monet, safe, wagner arnaut cto de tec..."
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,ceo da openai e diretora da ibm prestam depoim...,None,tecmasters.com.br,"{ibm, fbi, sony, google, openai, devs software...","{feira altman, montgomery, ted lieu, sam altma..."
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,ibm lancara llama da meta em plataforma de ia...,None,forbes.com.br,"{ibm, microsoft, google, openai, ibm watson}","{mark zuckerberg luta, watsonx}"
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,ibm lanca chip de ia analogico inspirado no ce...,None,olhardigital.com.br,"{ibm, ibm research, serie c0d3rs, ana luiza fi...","{chaosamran, ana luiza figueiredo}"


In [ ]:
teste_entities_df

,Entidades
0,"[(ibm, ORG), (watsonx, ORG), (bruno martinez, ..."
1,"[(ibm, ORG), (microsoft, ORG), (ibm consulting..."
2,"[(ibm brasil, ORG), (watson, PER), (watsonx, P..."
3,"[(ibm comentara jogos, ORG), (ibm, ORG), (esta..."
4,"[(ibm, ORG), (watson, PER), (watsonx, PER), (m..."
5,"[(overwatch, LOC), (ibm watson, PER), (thomas ..."
6,"[(bradesco, LOC), (ibm, ORG), (daniela frabasi..."
7,"[(openai, ORG), (ibm, ORG), (zac wolff unsplas..."
8,"[(ibm, ORG), (ibm, ORG), (ibm, ORG), (ibm, ORG..."
9,"[(ibm, ORG), (ibm, ORG), (ibm, ORG), (ana luiz..."


# 7.0 Aplicação das tecnicas de PLN

## 7.1 Stop Words

### 7.1.1 Definição da função

In [ ]:
def stopWords(dados):
    if isinstance(dados, str):
        dados = [dados]

    resultado = [' '.join([token.text for token in nlp(texto) if not token.is_stop]) for texto in dados]

    return resultado

### 7.1.2 Teste da função

In [ ]:
stopWords("um projeto realizado em parceria entre nasa e ibm pretende reduzir [...] em tempo quase que real")

['projeto realizado parceria nasa ibm pretende reduzir [ ... ] quase real']

In [ ]:
df_url['tag_removed'] = stopWords(df_url['tag_removed'])

## 7.2 Lematização

### 7.2.1 Definição da função

In [ ]:
def lematizar(dados):
    resultado = []
    for texto in dados:
        # Processa o texto com o modelo do Spacy
        doc = nlp(texto)
        # Lematiza o documento
        lemmas = [token.lemma_ for token in doc]
        # Junta tudo de volta a um texto
        textoLemmatizado = ' '.join(lemmas)
        resultado.append(textoLemmatizado)
    return resultado

### 7.2.2 Teste da função

In [ ]:
teste_string = [
    "projeto realizado parceria nasa ibm pretende reduzir tempo quase real"
]

lema_dataframe = pd.DataFrame(teste_string)

lema_dataframe[0] = lematizar(lema_dataframe[0])

In [ ]:
lema_dataframe

,0
0,projeto realizar parceria nasa ibm pretender r...


In [ ]:
df_url['tag_removed'] = lematizar(df_url['tag_removed'])

## 7.3 Tokenização

### 7.3.1 Definição da função

In [ ]:
def tokenizar_texto(texto):
    doc = nlp(texto)
    tokens = [token.text for token in doc]
    return tokens

### 7.3.2 Teste da função

In [ ]:
token_string = tokenizar_texto("projeto realizar parceria nasa ibm pretender reduzir tempo quase real")

In [ ]:
df_url['tag_removed'] = df_url['tag_removed'].apply(tokenizar_texto)

## 7.4 Dataframe resultante da seção

In [ ]:
df_url

,url_noticia,titulo,tag_removed,data,fonte,empresas,pessoas
0,https://www.showmetech.com.br/ibm-mostra-avanc...,"IBM fala sobre avanços da WatsonX, plataforma ...","[ibm, falar, avanco, Watsonx, plataformar, ai,...",None,www.showmetech.com.br,"{ibm, ibm brasil, halloween, bradesco, ibm ins...","{glauco vital, watsonx governance gestores, ma..."
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...,IBM e Microsoft expandem parceria para oferece...,"[ibm, microsoft, expander, parceria, oferecer,...",None,inforchannel.com.br,"{azure, ibm, ibm consulting azure openai servi...","{golpes infoblox, julius baer group, francesco..."
2,https://www.convergenciadigital.com.br/Negocio...,IBM Brasil quer levar base instalada de Watson...,"[ibm, Brasil, levar, base, instalar, watson, W...",None,www.convergenciadigital.com.br,"{ibm, ibm brasil, bradesco, watsonx, banco do ...","{foton, plataforma watson, watson assistant, m..."
3,https://inforchannel.com.br/2023/08/19/intelig...,Inteligência Artificial da IBM comentará jogos...,"[inteligencia, artificial, ibm, comentar, jogo...",None,inforchannel.com.br,"{ibm, ibm ix, ibm watsonx, infor channel, ibm ...","{red hat openshift, ibm watsonx powered, jonat..."
4,https://itforum.com.br/ibm-quer-ampliar-base-w...,IBM quer ampliar base de clientes Watson para ...,"[ibm, ampliar, base, cliente, watson, Watsonx,...",None,itforum.com.br,"{ibm, ibm brasil, ibm watsonx, ibm institute f...","{plataforma watson, computerworld cio, marcelo..."
5,https://www.tecmundo.com.br/voxel/219536-overw...,Overwatch League traz Hero Rankings com a IA I...,"[overwatch, league, trazer, hero, rankings, ir...",None,www.tecmundo.com.br,"{blizzard, ibm, ibm voce, cupom nike, epic gam...","{tiktok twitter, thomas schulze costa, contato..."
6,https://epocanegocios.globo.com/futuro-do-dinh...,Bradesco e Itaú se aliam a IBM para testar uso...,"[bradesco, itaur, al, ibm, testar, uso, comput...",None,epocanegocios.globo.com,"{ibm, editora globo, ibm brasil, bradesco, onu...","{wagner, monet, safe, wagner arnaut cto de tec..."
7,https://tecmasters.com.br/ceo-openai-diretora-...,CEO da OpenAI e diretora da IBM prestam depoim...,"[ceo, openai, diretora, ibm, prestam, depoimen...",None,tecmasters.com.br,"{ibm, fbi, sony, google, openai, devs software...","{feira altman, montgomery, ted lieu, sam altma..."
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...,IBM lançará Llama 2 da Meta em plataforma de IA,"[ibm, lancar, llama, , meta, plataforma, ir...",None,forbes.com.br,"{ibm, microsoft, google, openai, ibm watson}","{mark zuckerberg luta, watsonx}"
9,https://olhardigital.com.br/2023/08/22/ciencia...,IBM lança chip de IA analógico inspirado no cé...,"[ibm, lanca, chip, ir, analogico, inspirar, ce...",None,olhardigital.com.br,"{ibm, ibm research, serie c0d3rs, ana luiza fi...","{chaosamran, ana luiza figueiredo}"


# 8.0 Pipeline de pré processamento

## 8.1 Importação de um novo dataframe

In [ ]:
df_pipeline = pd.read_csv('/content/drive/Shareddrives/CogniVoice/ibm_db.csv')

In [ ]:
df_pipeline

,url_noticia
0,https://www.showmetech.com.br/ibm-mostra-avanc...
1,https://inforchannel.com.br/2023/08/17/ibm-e-m...
2,https://www.convergenciadigital.com.br/Negocio...
3,https://inforchannel.com.br/2023/08/19/intelig...
4,https://itforum.com.br/ibm-quer-ampliar-base-w...
5,https://www.tecmundo.com.br/voxel/219536-overw...
6,https://epocanegocios.globo.com/futuro-do-dinh...
7,https://tecmasters.com.br/ceo-openai-diretora-...
8,https://forbes.com.br/forbes-tech/2023/08/ibm-...
9,https://olhardigital.com.br/2023/08/22/ciencia...


## 8.2 Definição da função

In [ ]:
def pipeline(dados, colum_name):
  #Cria um dataframe vazio para armazenar os dados tratados
  treated_data =  pd.DataFrame()

  #Busca pela url o site, faz o tratamento do html e retorna o texto da noticia
  treated_data["data"] = dados[colum_name].apply(remove_tags)

  treated_data["data"] = treated_data["data"].apply(remove_duplicate_spaces)

  #Remove possiveis acentos da tabela
  treated_data = treated_data.applymap(remocaoAcentos)

  treated_data = treated_data.applymap(lowercase_text)

  treated_data["data"] = stopWords(treated_data["data"])

  treated_data["data"] = lematizar(treated_data["data"])

  treated_data["data"] = treated_data["data"].apply(tokenizar_texto)

  return treated_data


## 8.3 Teste da função

In [ ]:
teste_pipeline = pipeline(df_pipeline, "url_noticia")

## 8.4 Dataframe resultante da seção

In [ ]:
teste_pipeline

,data
0,"[ibm, falar, avanco, Watsonx, plataformar, ai,..."
1,"[ibm, microsoft, expander, parceria, oferecer,..."
2,"[ibm, Brasil, levar, base, instalar, watson, W..."
3,"[inteligencia, artificial, ibm, comentar, jogo..."
4,"[ibm, ampliar, base, cliente, watson, Watsonx,..."
5,"[overwatch, league, trazer, hero, rankings, ir..."
6,"[bradesco, itaur, al, ibm, testar, uso, comput..."
7,"[ceo, openai, diretora, ibm, prestam, depoimen..."
8,"[ibm, lancar, llama, meta, plataformo, ir, emp..."
9,"[ibm, lanca, chip, ir, analogico, inspirar, ce..."


In [ ]:
teste_pipeline = maiusculas(teste_pipeline)

In [ ]:
teste_pipeline

,data
0,"[ibm, falar, avanco, Watsonx, plataformar, ai,..."
1,"[ibm, microsoft, expander, parceria, oferecer,..."
2,"[ibm, Brasil, levar, base, instalar, watson, W..."
3,"[inteligencia, artificial, ibm, comentar, jogo..."
4,"[ibm, ampliar, base, cliente, watson, Watsonx,..."
5,"[overwatch, league, trazer, hero, rankings, ir..."
6,"[bradesco, itaur, al, ibm, testar, uso, comput..."
7,"[ceo, openai, diretora, ibm, prestam, depoimen..."
8,"[ibm, lancar, llama, meta, plataformo, ir, emp..."
9,"[ibm, lanca, chip, ir, analogico, inspirar, ce..."


# 9.0 Bag of Words

## 9.1 Definição das funções isoladas

In [ ]:
# função para vetor de recorrencia

def bagOfWords(dicionario,dados):
  salvar = dicionario.texts_to_matrix(dados, mode='count')
  return salvar

In [ ]:
def criar_dicionario(dados):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(dados)
  return tokenizer

## 9.2 Teste individual das funções

In [ ]:
teste_bow = [
    "projeto,realizar,parceria,nasa,ibm,pretender,reduzir,tempo,quase,real",
    "projeto,realizar,parceria,microsoft,ibm,inteligencia,artificial,generativa,nuvem"
]

bow_dataframe = pd.DataFrame(teste_bow)

In [ ]:
dicionarioTeste = criar_dicionario(bow_dataframe[0])
print(dicionarioTeste)
print(list(dicionarioTeste.word_index.keys()))
print(len(list(dicionarioTeste.word_index.keys())))

['projeto', 'realizar', 'parceria', 'ibm', 'nasa', 'pretender', 'reduzir', 'tempo', 'quase', 'real', 'microsoft', 'inteligencia', 'artificial', 'generativa', 'nuvem']
15


In [ ]:
dicionarioTeste2 = criar_dicionario(teste_pipeline["data"])
print(dicionarioTeste2)
print(list(dicionarioTeste2.word_index.keys()))
print(len(list(dicionarioTeste2.word_index.keys())))

['ibm', 'ir', 'tecnologia', 'empresa', 'ha', 'tambem', 'watsonx', 'ser', 'usp', 'ano', 'inteligencia', 'artificial', 'modelo', 'brasil', 'dar', 'quantico', 'ajudar', 'nao', 'dado', 'generativo', 'uso', 'digital', 'sao', 'hora', 'negocio', 'cupom', 'ja', 'computacao', 'noticio', 'cliente', 'ai', 'mercado', 'projeto', 'trabalho', 'watson', 'politico', 'estao', 'mail', 'brasileiro', 'minuto', 'computador', 'processo', 'ate', 'aler', 'privacidade', 'microsoft', 'jogo', 'mundo', 'voce', 'estar', 'incluir', 'consulting', 'abrir', 'nasa', 'mostrar', 'caso', 'plataforma', 'analise', 'bom', 'chip', 'servico', 'conteudo', 'inovacao', 'tecmur', 'possivel', 'pesquisa', 'us', 'foto', 'fazer', 'anunciar', 'setor', 'falar', 'principal', 'direito', 'parceria', 'criar', 'trabalhar', 'ti', 'indigena', 'resultado', 'realizar', 'dia', 'de', 'oferecer', 'ceo', 'c', 's', 'paulo', 'video', 'rede', 'imagem', 'saude', 'bradesco', 'data', 'utilizar', 'azure', 'banco', 'acordo', 'lider', 'seguranca', 'linguagem'

In [ ]:
bagOfWords(dicionarioTeste,bow_dataframe[0])

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])

In [ ]:
bagOfWords(dicionarioTeste2,teste_pipeline["data"])

array([[ 0., 20.,  8., ...,  0.,  0.,  0.],
       [ 0., 19., 19., ...,  0.,  0.,  0.],
       [ 0., 17., 19., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  9.,  4., ...,  0.,  0.,  0.],
       [ 0., 19., 17., ...,  0.,  0.,  0.],
       [ 0.,  7.,  0., ...,  1.,  1.,  1.]])

## 9.3 Definição e criação do modelo

In [ ]:
#função de vetorização de Bag of words
def Bow(dados):
  #Recebe os dados a ser realizado o BOW
  dadosTratados = dados
  dadosTratados =  pd.DataFrame({'dadosBow': dadosTratados})
  #geração de dicionario
  dicionario = criar_dicionario(dadosTratados['dadosBow'])
  # chamar o modelo de bag of words
  modelo = (bagOfWords(dicionario,dadosTratados['dadosBow']))
  return modelo,dicionario

## 9.4 Teste do modelo BoW

In [ ]:
#salvando e exibindo resultados
vetor,dicionario = Bow(teste_pipeline["data"])
dicionario = list(dicionario.word_index.keys())
print(dicionario)
print(vetor)
print(vetor[:, 0])

['ibm', 'ir', 'tecnologia', 'empresa', 'ha', 'tambem', 'watsonx', 'ser', 'usp', 'ano', 'inteligencia', 'artificial', 'modelo', 'brasil', 'dar', 'quantico', 'ajudar', 'nao', 'dado', 'generativo', 'uso', 'digital', 'sao', 'hora', 'negocio', 'cupom', 'ja', 'computacao', 'noticio', 'cliente', 'ai', 'mercado', 'projeto', 'trabalho', 'watson', 'politico', 'estao', 'mail', 'brasileiro', 'minuto', 'computador', 'processo', 'ate', 'aler', 'privacidade', 'microsoft', 'jogo', 'mundo', 'voce', 'estar', 'incluir', 'consulting', 'abrir', 'nasa', 'mostrar', 'caso', 'plataforma', 'analise', 'bom', 'chip', 'servico', 'conteudo', 'inovacao', 'tecmur', 'possivel', 'pesquisa', 'us', 'foto', 'fazer', 'anunciar', 'setor', 'falar', 'principal', 'direito', 'parceria', 'criar', 'trabalhar', 'ti', 'indigena', 'resultado', 'realizar', 'dia', 'de', 'oferecer', 'ceo', 'c', 's', 'paulo', 'video', 'rede', 'imagem', 'saude', 'bradesco', 'data', 'utilizar', 'azure', 'banco', 'acordo', 'lider', 'seguranca', 'linguagem'

## 9.5 Visualização do modelo

In [ ]:
#vetor
vetorexportação = vetor[:, 1:]
#para conseguir ver e exportar
dfBOW = pd.DataFrame(vetorexportação, columns=dicionario, index=teste_pipeline)
dfBOW

,ibm,ir,tecnologia,empresa,ha,tambem,watsonx,ser,usp,ano,...,atrasa,traje,pouso,ingenuity,10o,voo,brother,voluntario,simulacao,imitar
"([ibm, falar, avanco, Watsonx, plataformar, ai, empresa, bruno, Martinez, agosto, empresa, mostrar, resultado, bia, assistente, pessoal, bradesco, comentar, inteligencia, artificial, ja, ajudar, empresa, parceiro, indice, ibm, ajudar, empresa, entrar, mundo, ai, Watsonx, Watsonx, data, Watsonx, governance, gestor, enfrentar, desafio, ligar, gestao, dar, empresa, durante, evento, fechar, jornalista, ibm, falar, importanciar, inteligencia, artificial, generativo, tambem, chamar, genai, ajudar, empresa, diversos, mercado, especifico, Watsonx, utilizar, diversos, caso, sucesso, bradesco, desenvolver, assistente, pessoal, bia, detalhe, ibm, ajudar, empresa, entrar, mundo, ai, empresa, ser, solicitar, ajudar, hardware, estao, ser, contratar, tambem, aplicacao, inteligencia, artificial, diversos, processo, caso, voce, nao, saber, ir, generativa, utilizar, diversos, ...],)",20.0,8.0,6.0,24.0,0.0,17.0,14.0,10.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([ibm, microsoft, expander, parceria, oferecer, solucoes, ir, generativo, oferta, ibm, consulting, Azure, openai, service, disponivel, azure, marketplace, servico, ir, totalmente, gerenciar, redacao, agosto, pm, compartilhar, ibm, anunciar, feira, expandir, colaboracao, microsoft, ajudar, cliente, conjunto, acelerar, implementacao, ir, generativa, oferecer, oferta, fornecer, cliente, experiencia, tecnologia, precisar, inovar, processo, negocio, dimensionar, ir, maneira, eficaz, ibm, consulting, colaboracao, microsoft, concentrar, ajudar, cliente, implementar, dimensionar, azure, openai, service, oferto, ibm, consulting, Azure, openai, service, disponivel, azure, marketplace, servico, ir, totalmente, gerenciar, permitir, desenvolvedore, cientista, dado, apliquir, modelo, linguagem, poderoso, incluir, serie, gpt, codex, objetivo, ajudar, empresa, definir, estrategia, adocao, conjunto, inicial, caso, uso, ir, ...],)",19.0,19.0,14.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([ibm, Brasil, levar, base, instalar, watson, Watsonx, plataforma, dar, ir, roberto, prescott, convergencia, digital, ibm, Brasil, base, instalar, cliente, usar, Watson, habilitar, ecossistema, dar, inteligencia, artificial, Watsonx, nao, tratar, substituicao, upgrade, conforme, empresa, explicar, complementaridade, ampliar, funcionalidade, ja, existente, acordo, ibm, existir, projeto, Watson, Brasil, Corporacoes, bradesco, banco, Brasil, startups, Brasil, destacar, adocao, Watson, centena, caso, implantado, entao, base, relevante, estar, fazer, trabalho, expandir, complementar, plataforma, Watson, Watsonx, trazer, diferente, trabalhar, dado, subsidiar, inteligencia, artificial, apontar, Marcelo, Braga, presidente, lider, technology, ibm, Brasil, conversa, jornalista, feira, durante, ibm, think, sao, Paulo, Watsonx, disponibilizar, julho, apo, ser, lancar, maio, durante, think, ...],)",17.0,19.0,3.0,4.0,2.0,1.0,11.0,6.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"([inteligencia, artificial, ibm, comentar, jogo, torneio, us, Open, tenis, longo, torneio, tecnologia, ir, automatizara, producao, narracoes, audio, detalhar, legenda, acompanhar, video, bom, momento, torneio, redacao, agosto, am, compartilhar, ibm, associacao, tenis, Estados, Unidos, ust, anunciar, funcionalidade, digital, Aparecerao, usopen, org, aplicativo, us, Open, fas, incluir, comentario, falar, gerar, inteligencia, artificial, ir, desenvolver, oferecer, fas, experiencia, perspicaz, engajadora, atualizar, momento, importante, torneio, ibm, Watsonx, powered, ai, commentary, modelo, construir, treinar, implementado, Watsonx, plataforma, dar, ir, ibm, longo, torneio, tecnologia, ir, automatizara, producao, narracoes, audio, detalhar, legenda, acompanhar, video, bom, momento, us, Open, escalar, precedente, partida, sorteio, individual, dezessete, quadra, implementar, ibm, ...],

In [ ]:
def contarPalavras(dicionario, matriz):
    dicionario = [''] + dicionario
    contagemPalavras = [0] * len(dicionario)

    for i, palavra in enumerate(dicionario):
        for j in range(len(matriz)):
            contagemPalavras[i] += matriz[j][i]

    contagemDf = pd.DataFrame({'palavra': dicionario, 'repetição': contagemPalavras})

    return contagemDf.iloc[1:]

In [ ]:
# Geração da tabela de contagem
contador = contarPalavras(dicionario, vetor)

In [ ]:
# Exibição das primeiras 10 palavras mais recorrentes
contador.head(10)

,palavra,repetição
1,ibm,220.0
2,ir,163.0
3,tecnologia,98.0
4,empresa,85.0
5,ha,84.0
6,tambem,61.0
7,watsonx,60.0
8,ser,60.0
9,usp,60.0
10,ano,57.0


# 10.0 TF-IDF (TBD)

Frequência do termo (TF) – é a frequência das palavras em um texto de amostra.

Frequência inversa do documento (IDF) – destaca a frequência das palavras em outros textos de amostra. Os recursos são raros ou comuns nos textos de exemplo é a principal preocupação aqui.

Quando usamos ambos os TF-IDF juntos (TF * IDF), as palavras de alta frequência em um texto de exemplo que tem baixa ocorrência em outros textos de exemplo recebem maior importância.

# 11.0 Word2vec (TBD)

## 11.1 CBOW (Continuous Bag of Words) (TBD)

## 11.2 Skipgram (TBD)

# 12.0 Similaridade do coseno (TBD)

# 13.0 Flask API (TBD)